# Automated Essay Scoring
Machine Learning Challenge by [Kaggle](https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2/overview)

In [187]:
import pandas as pd

In [188]:
df = pd.read_csv('data/train.csv')

In [189]:
df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\r\n\r\nThis is a letter to...",3


In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   17307 non-null  object
 1   full_text  17307 non-null  object
 2   score      17307 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 405.8+ KB


In [191]:
df.describe()

,score
count,17307.000000
mean,2.948402
std,1.044899
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,6.000000


In [192]:
df.isna().sum()

essay_id     0
full_text    0
score        0
dtype: int64

In [193]:
df['score'].value_counts()

score
3    6280
2    4723
4    3926
1    1252
5     970
6     156
Name: count, dtype: int64

In [194]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='score'), df['score'], test_size=0.2, random_state=42)

## Evaluation using the Quadratic Weighted Kappa
The quadratic weighted kappa (QWK) score ranges from -1 to 1. A score of 1 indicates perfect agreement between the predicted score and the true score. A score of 0 indicates the agreement is no better than random. A score of -1 indicates perfect disagreement between the predicted score and the true score.

In [195]:
from sklearn.metrics import confusion_matrix

def quadratic_weighted_kappa(y_true, y_pred, min_rating=None, max_rating=None):
    """
    Computes the quadratic weighted kappa.
    """
    if min_rating is None:
        min_rating = min(min(y_true), min(y_pred))
    if max_rating is None:
        max_rating = max(max(y_true), max(y_pred))

    conf_mat = confusion_matrix(y_true, y_pred, labels=range(min_rating, max_rating + 1))
    num_ratings = len(conf_mat)
    num_scored_items = float(len(y_true))

    hist_true = np.histogram(y_true, bins=np.arange(min_rating, max_rating + 2))[0]
    hist_pred = np.histogram(y_pred, bins=np.arange(min_rating, max_rating + 2))[0]

    expected_mat = np.outer(hist_true, hist_pred) / num_scored_items

    weight_mat = np.zeros((num_ratings, num_ratings))
    for i in range(num_ratings):
        for j in range(num_ratings):
            weight_mat[i, j] = ((i - j) ** 2) / ((num_ratings - 1) ** 2)

    kappa = 1.0 - (np.sum(weight_mat * conf_mat) / np.sum(weight_mat * expected_mat))
    return kappa

## Using CountVectorizer and TfidfTransformer from sklearn 
[Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

*Simple Text Preprocessing*

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train['full_text'])
X_train_counts.shape

(13845, 56588)

Literally a word count but with tokens/chunks of texts. 

In [197]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(13845, 56588)

TfidfTransformer is used to convert the word count into a frequency matrix. We simply divide the number of word per document/data/observation by total number of this word in all documents.

In [198]:
y_train.shape

(13845,)

In [199]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [200]:
X_new_counts = count_vect.transform(X_test['full_text'])
X_new_counts

<3462x56588 sparse matrix of type '<class 'numpy.int64'>'
	with 564659 stored elements in Compressed Sparse Row format>

In [201]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
X_new_tfidf

<3462x56588 sparse matrix of type '<class 'numpy.float64'>'
	with 564659 stored elements in Compressed Sparse Row format>

In [202]:
predicted = clf.predict(X_new_tfidf)
predicted

array([3, 3, 3, ..., 3, 3, 3])

In [203]:
pred_df = pd.DataFrame({'essay_id': X_test['essay_id'], 'full_text': X_test['full_text'], 'score': predicted})
pred_df

,essay_id,full_text,score
12696,bb4c434,"People tend to use there cars so much, they ba...",3
4625,44e88b0,Imagine being a top scientist at NASA and Viki...,3
733,0ba78ec,The face of Mars could not be created by alien...,3
16885,f96c287,Many people belive that the face on Mars was c...,3
3334,317173f,Driverless Cars are coming soon or later? Peop...,3
...,...,...,...
16145,ee1d27b,How the author support his suggests that study...,3
4229,3e7dd0b,"In this aricle , the author its trying to you ...",3
4313,3fdbec2,The Facial Action Coding System enables comput...,3
934,0edee1b,"Hello my name is Luke Bomberger and, welcome t...",3


In [204]:
import numpy as np
print(f"Mean: {np.mean(predicted == y_test)}")

Mean: 0.3694396302715193


In [205]:
kappa_score = quadratic_weighted_kappa(y_test, predicted)
print(f"Quadratic Weighted Kappa: {kappa_score}")

Quadratic Weighted Kappa: 0.034102332855281525


In [206]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train['full_text'], y_train)
predicted = text_clf.predict(X_test['full_text'])

print(f"Mean: {np.mean(predicted == y_test)}")

kappa_score = quadratic_weighted_kappa(y_test, predicted)
print(f"Quadratic Weighted Kappa: {kappa_score}")

Mean: 0.4679376083188908
Quadratic Weighted Kappa: 0.5708430116379231


Not so bad, getting a score of 0.57

For my next steps: 
<br/>
https://www.datacamp.com/blog/what-is-tokenization
<br/>
https://www.nltk.org
<br/>
https://www.nyckel.com/pretrained-classifiers/
<br/>

## Using [NLTK](https://www.nltk.org/)
Following [NLP Tutorial for Text Classification in Python](https://medium.com/analytics-vidhya/nlp-tutorial-for-text-classification-in-python-8f19cd17b49e)
<br/>
Some [documentation](https://datascientest.com/nlp-word-embedding-word2vec) 

In [207]:
# Text preprocessing
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# for word embedding*
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /home/jacq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jacq/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jacq/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jacq/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [208]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [209]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

X_train['clean_text'] = X_train['full_text'].apply(lambda x: finalpreprocess(x))
X_train.head()

,essay_id,full_text,clean_text
12931,be29f81,There is no question that the driverless car i...,question driverless car technological advancem...
12955,be96267,"I will be telling you about a story called ""A ...",tell story call cowboy rode wave think good st...
4410,417a337,While driveless cars a cooler driving yourself...,driveless car cooler drive may bite safe every...
15013,dcf3a1e,"Dear State Senator,\r\n\r\nI do not like the E...",dear state senator like electoral college elec...
10643,9d04b4e,"The "" Unmasking the Face on Mars"" was NOT crea...",unmask face mar create alien face mar natural ...


In [210]:
X_test['clean_text'] = X_test['full_text'].apply(lambda x: finalpreprocess(x))
X_test.head()

,essay_id,full_text,clean_text
12696,bb4c434,"People tend to use there cars so much, they ba...",people tend use car much basically live get da...
4625,44e88b0,Imagine being a top scientist at NASA and Viki...,imagine top scientist nasa viking spacecraft s...
733,0ba78ec,The face of Mars could not be created by alien...,face mar could create alien land form create c...
16885,f96c287,Many people belive that the face on Mars was c...,many people belive face mar create alien alien...
3334,317173f,Driverless Cars are coming soon or later? Peop...,driverless car come soon later people argue po...


In [211]:
# Tokenize the sentences
X_train['clean_text_tok'] = X_train['clean_text'].apply(nltk.word_tokenize)
X_test['clean_text_tok'] = X_test['clean_text'].apply(nltk.word_tokenize)

In [212]:
X_test.head()

,essay_id,full_text,clean_text,clean_text_tok
12696,bb4c434,"People tend to use there cars so much, they ba...",people tend use car much basically live get da...,"[people, tend, use, car, much, basically, live..."
4625,44e88b0,Imagine being a top scientist at NASA and Viki...,imagine top scientist nasa viking spacecraft s...,"[imagine, top, scientist, nasa, viking, spacec..."
733,0ba78ec,The face of Mars could not be created by alien...,face mar could create alien land form create c...,"[face, mar, could, create, alien, land, form, ..."
16885,f96c287,Many people belive that the face on Mars was c...,many people belive face mar create alien alien...,"[many, people, belive, face, mar, create, alie..."
3334,317173f,Driverless Cars are coming soon or later? Peop...,driverless car come soon later people argue po...,"[driverless, car, come, soon, later, people, a..."


In [213]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True) # WordVectorizer and TfidfTransformer in one model
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train['clean_text'])
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test['clean_text'])

In [214]:
#building Word2Vec model

model = Word2Vec(sentences=X_train['clean_text_tok'], 
                 vector_size=100, 
                 window=5, 
                 min_count=1, 
                 workers=4
                 )
w2v = {word: vector for word, vector in zip(model.wv.index_to_key, model.wv.vectors)}

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0) for words in X
        ])

In [215]:
modelw = MeanEmbeddingVectorizer(w2v)
# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train['clean_text_tok'])
X_test_vectors_w2v = modelw.transform(X_test['clean_text_tok'])

In [216]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', penalty='l2',
                    alpha=1e-3, random_state=42,
                    max_iter=5, tol=None)

clf.fit(X_train_vectors_tfidf, y_train)
predicted = clf.predict(X_test_vectors_tfidf)

print(f"Mean: {np.mean(predicted == y_test)}")

kappa_score = quadratic_weighted_kappa(y_test, predicted)
print(f"Quadratic Weighted Kappa: {kappa_score}")

Mean: 0.45031773541305603
Quadratic Weighted Kappa: 0.5574675462818004


## NLP with [Keras pretrained models](https://keras.io/api/keras_nlp/models/#kerasnlp-models)

[Bert Classifiers](https://keras.io/api/keras_nlp/models/bert/bert_classifier/)

In [276]:
import keras_nlp
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased", num_classes=7, preprocessor=None)

In [277]:
# Convert to TensorFlow dataset
def dataframe_to_dataset(X, y):
    df = X.copy()
    df['labels'] = y
    ds = tf.data.Dataset.from_tensor_slices((dict(df)))
    return ds

train_tf_dataset = dataframe_to_dataset(X_train[['clean_text']], y_train)
test_tf_dataset = dataframe_to_dataset(X_test[['clean_text']], y_test)

In [278]:
def get_texts_and_labels(X, y):
    texts = X['clean_text'].tolist()
    labels = y.tolist()
    return texts, labels

train_texts, train_labels = get_texts_and_labels(X_train[['clean_text']], y_train)
test_texts, test_labels = get_texts_and_labels(X_test[['clean_text']], y_test)

print((train_texts[0], train_labels[0]))

('question driverless car technological advancement today society however importanat question yet completely answered question full use driverless car necessary technological advancement answer yes important question driverless car necessary technological advancement first reason infinite possibility taht driverless car offer would benefit society second reason safety driverless car could offer complete fully reason support driverless car could benefit society many way technological advancement driverless car necessary one would benefit society developmentment driverless car would offer infinite possiblities would provide many important benefit one benefit driver would ability rest relax may argue driver never rest relax wheel however driverless car completely develop wouldnt need driver car drive driver relaxes well alternative tire sriver also mention passage every car driverless transportation efficiency speed would excel every fleet car system transportation would efficient flawles

In [279]:
# Preprocess the texts 
# The preprocessor will tokenize the texts, and encode them into ids to map to the BERT vocabulary
preprocessed_train_texts = preprocessor(train_texts)
preprocessed_test_texts = preprocessor(test_texts)

In [281]:
classifier.fit(x=preprocessed_train_texts, y=train_labels, batch_size=2)

6923/6923 ━━━━━━━━━━━━━━━━━━━━ 724s 105ms/step - loss: 1.1190 - sparse_categorical_accuracy: 0.5545


In [282]:
import keras


classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    jit_compile=True,
)
classifier.fit(x=preprocessed_train_texts, y=train_labels, batch_size=2)

I0000 00:00:1716581193.512447  349193 service.cc:145] XLA service 0x7f438825d680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716581193.514981  349193 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-05-24 22:06:33.795496: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


   1/6923 ━━━━━━━━━━━━━━━━━━━━ 21:58:47 11s/step - loss: 0.6054 - sparse_categorical_accuracy: 1.0000

I0000 00:00:1716581199.672713  349193 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6923/6923 ━━━━━━━━━━━━━━━━━━━━ 639s 91ms/step - loss: 0.8801 - sparse_categorical_accuracy: 0.6256


In [285]:
predictions = classifier.predict(preprocessed_test_texts, batch_size=2)

1731/1731 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step


array([[-7.01515   , -2.5966995 , -1.4228523 , ...,  2.6925843 ,
         0.9004487 , -2.8749375 ],
       [-5.622581  , -2.668289  , -1.8633065 , ...,  2.8382761 ,
         1.638929  , -1.840886  ],
       [-7.6777434 , -1.1952523 ,  1.6521977 , ..., -0.02198903,
        -3.3243122 , -5.6559286 ],
       ...,
       [-6.1243434 ,  1.3501959 ,  1.371524  , ..., -0.8509998 ,
        -2.9880426 , -3.949362  ],
       [-6.051654  ,  1.4242927 ,  3.19057   , ..., -2.1929855 ,
        -3.2725418 , -4.083208  ],
       [-6.790829  ,  0.6585024 ,  3.355659  , ..., -1.9247309 ,
        -3.2917526 , -4.6624737 ]], dtype=float32)

In [287]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes

array([4, 4, 3, ..., 2, 2, 2])

In [288]:
print(f"Mean: {np.mean(predicted_classes == y_test)}")

print(f"Quadratic Weighted Kappa: {quadratic_weighted_kappa(y_test, predicted_classes)}")

Mean: 0.5990756787983824
Quadratic Weighted Kappa: 0.7629359037697089


pip install pipreqs
pipreqs 